# Rappels jour 2 

* gotchas sur les fonctions 
    * pas d'arguments mutables en entré
    * accès aux varaible : L(ocal) E(nglobing) G(lobal) B(uiltin)
    * les variables englobing sont passées par closure
    * en python les closures : 
        * sont late binding 
        * sont en lecture seule
* curryfication =>  passer d'une fonction à N arguments à N fonctions imbriquées à 1 argument chacune 
* décorateurs
    * on peut le comprendre comme une composée de fonction `toto = deco(toto)` 
    * on peut chainer les décorateurs, l'ordre a une importance 
    * la fonction a l'intérieur s'appelle `wrapper` par convention 
    * dans le cas général, la bonne pratique consiste à décorer le wrapper avec `functools.wraps` pour conserver les métadonnées de la fonction d'origine (nom, docstring)
* les décorateurs peuvent être paramétrés, on rajoute une fonction qui va générer le décorateur qui nous intéresse

In [9]:
class Toto:
    def __init__(self, a):
        self.a = a 
    
    def hello(self):
        print(self.a)
    
    def __call__(self, b):
        print(b)
        
        
t = Toto("a")
t("b")

b
None


In [2]:
import inspect

In [3]:
inspect.getsourcefile(Toto)

TypeError: <class '__main__.Toto'> is a built-in class

In [4]:
inspect.getsourcelines(Toto)

TypeError: <class '__main__.Toto'> is a built-in class

In [23]:
from typing import Callable, TypeVar

T = TypeVar('T')

class MyDecorator:
    my_list = []
    
    def __init__(self, func: Callable[[T], T]):
        self.func = func
        self.my_list.append(func)
        
    def __call__(self, *args, **kwargs) -> T:
        print("Before function call")
        result = self.func(*args, **kwargs)
        print("After function call")
        return result
    
    def analyse(self):
        self.my_list

In [24]:
@MyDecorator
def my_function(arg: str) -> str:
    return arg.upper()


@MyDecorator
def toto(arg: str) -> str:
    return arg.upper()

my_function("hello")

Before function call
After function call


'HELLO'

In [25]:
MyDecorator.my_list

[<function __main__.my_function(arg: str) -> str>,
 <function __main__.toto(arg: str) -> str>]

# Gestion de fichiers / gestionnaires de contexte

In [54]:
try:
    f = open("./fibuzz.py")
except FileNotFoundError:
    print("le fichier n'est pas trouvé")
finally:
    try:
        f.close()
    except NameError:
        pass

le fichier n'est pas trouvé


In [49]:
f.closed

True

In [38]:
try:
    import numpi 
except ImportError as e:
    print(e, type(e))

numpi.array()

No module named 'numpi' <class 'ModuleNotFoundError'>


NameError: name 'numpi' is not defined

In [55]:
def exemple():
    try:
        print("dans try")
        return 1
    finally:
        return 2
    
exemple()

dans try


2

In [56]:
with open("./fizzbuzz.py") as f_text:
    1 / 0

ZeroDivisionError: division by zero

In [57]:
f_text.closed

True

In [67]:
class MonContextManager:
    def __enter__(self):
        print("dans enter")
        return "toto"

    def __exit__(self, exception_type, exception_object, traceback):
        print("dans exit", exception_type, exception_object, traceback)
    
mcm = MonContextManager()

with mcm as f:
    print("dans le with", f)

dans enter
dans le with toto
dans exit None None None


In [72]:
import time

class MonContextManager:
    def __enter__(self):
        self.tic = time.time()

    def __exit__(self, exception_type, exception_object, traceback):
        print(time.time() - self.tic)
    
mcm = MonContextManager()

with mcm as f:
    time.sleep(1)

1.0002660751342773


In [68]:
from contextlib import contextmanager

@contextmanager
def hello_context_manager():
    print("Entering the context...")
    yield "Hello, World!"
    print("Leaving the context...")


with hello_context_manager() as hello:
    print(hello)


Entering the context...
Hello, World!
Leaving the context...


# Rappel matin

* fin des décorateurs
    * sous forme de classe => surcharger `__call__`
    * code à l'intérieur du wrapper éxécuté lors de l'appel de la fonction, code à l'extérieur exécuté au moment de la décoration de la fonction 
* context manager
    * à quoi ça sert ? isoler un environemment / un contexte dans un bloc, il se passe quelque chose à l'entrée et à la sortie du bloc 
    * restaurer un état même si le code à l'intérieur du bloc à crashé (fermer un fichier, relacher un verrou)
    * comment ça s'écrit ?
        * `with ... (as ...):`
        * implémenter sur n'importe quelle classe 2 méthodes : `__enter__` et `__exit__`
        * `@contextlib.contextmanager` => il faut qu'il y ait **un** `yield` (la fonction doit être un générateur), ce qui est avant sera dans le `__enter__` et après dans le `__exit__`
* POO
    * variables de classes
    * rappels POO

In [12]:
class Exemple:
    def __init__(self):
        print(self)
        self.toto = "kegtrht"

e = Exemple()
print(e, type(e), hex(id(e)))

e.toto = "rtyjh"
print(e.toto)

<__main__.Exemple object at 0x7fb4d43e4b50> <class '__main__.Exemple'> 0x7fb4d43e4b50
rtyjh


In [14]:
e2 = Exemple()
print(e2)

print(e, e2)

<__main__.Exemple object at 0x7fb4d43e4b50> <__main__.Exemple object at 0x7fb4b680fd60>


In [18]:
class Jolie:
    def __str__(self):
        return f"je suis un objet de type {type(self)}"

j = Jolie()
print(j)

je suis un objet de type <class '__main__.Jolie'>


# Copie de variables

In [20]:
a = []
b = a

a.append(1)
print(id(a), id(b))
print(a, b)

140414135172416 140414135172416
[1] [1]


In [21]:
a = []
b = a[:]

a.append(1)
print(id(a), id(b))
print(a, b)

140414136227072 140414135172736
[1] []


In [22]:
from copy import copy

a = []
b = copy(a)

a.append(1)
print(id(a), id(b))
print(a, b)

140414135146304 140414135146176
[1] []


In [31]:
a = [[]]
b = copy(a)
a.append("toto")
a[0].append(1)
print(id(a), id(b))
print(id(a[0]), id(b[0]))
print(a, b)

140414135203584 140414134908864
140414134884480 140414134884480
[[1], 'toto'] [[1]]


In [32]:
from copy import deepcopy

a = [[]]
b = deepcopy(a)
print(id(a), id(b))
print(id(a[0]), id(b[0]))

a[0].append(1)
print(a, b)

140414134869440 140414134819648
140414134935808 140414134918528
[[1]] [[]]


In [37]:
class Tout:
    def __init__(self):
        self.salut = "oui"
        
    def __getattr__(self, name):
        print("on demande name")
        return "non"
    
t = Tout()
print(t.salut)
print(t.bof)

oui
on demande name
non


In [40]:
class TemperatureConverter:
    def __init__(self, kelvin):
        self.kelvin = kelvin
    
    @property
    def celsius(self):
        print("dans celsius get")
        return self.kelvin - 273.15
    
    @celsius.setter
    def celsius(self, value_celsius):
        print("dans celsius set", value_celsius)
        self.kelvin = value_celsius + 273.15

In [41]:
tc = TemperatureConverter(10)

In [42]:
tc.kelvin

10

In [43]:
tc.kelvin = 30

In [44]:
tc.celsius

dans celsius get


-243.14999999999998

In [45]:
tc.celsius = 30

dans celsius set 30


In [46]:
tc.kelvin

303.15

In [52]:
class Exemple:
    def methode(self):
        print(self, type(self))
        current_type = type(self)
        return current_type()
        
e = Exemple()
e2 = e.methode()
print(e2)

<__main__.Exemple object at 0x7fb4b68b4af0> <class '__main__.Exemple'>


In [55]:
(type(4))(345.32)

345